In [30]:
#!pip install python-dotenv

In [29]:
from dotenv import load_dotenv
import os
import requests
import csv
import time

In [ ]:
# You can create a .env file in any directory for your project on Windows or MacOS systems
# %%writefile is a magic command in Google Colab (and Jupyter) that allows you to write the content of a cell directly to a file
# You'll need to replace XXX below with your Finnhub API key
# Please delete this cell after you create your .env file
%%writefile "./.env"
API_KEY="xxx"

UsageError: Line magic function `%%writefile` not found.


In [31]:
# Load the .env file
dotenv_path = "./.env"
load_dotenv(dotenv_path)
# Access the variables
API_KEY = os.getenv('API_KEY')

In [189]:
company = "ntap"
year = 2025
quarter = 1
dataRequest = requests.get("https://finnhub.io/api/v1/stock/financials-reported", params= {'symbol': company,'freq': 'quarterly','token':API_KEY}).json()
if 'data' in dataRequest:
    for report in dataRequest['data']:
        if report["year"] == year and report["quarter"] == quarter:
            for item in report['report']['bs']:
                if item['concept'] == "us-gaap_Assets":
                    print(item)
else:
    print(dataRequest)


{'concept': 'us-gaap_Assets', 'unit': 'u_usd', 'label': 'Assets', 'value': 9679000000}
{'concept': 'us-gaap_Assets', 'unit': 'u_usd', 'label': 'Total assets', 'value': 9311000000}


In [ ]:
def get_quarterly_total_assets(api_key, company, year, quarter):
    dataRequest = requests.get("https://finnhub.io/api/v1/stock/financials-reported", params= {'symbol': company,'freq': 'quarterly','token':API_KEY}).json()
    time.sleep(1)  # to respect the API rate limit
    if 'data' in dataRequest:
        for report in dataRequest['data']:
            if report["year"] == year and report["quarter"] == quarter:
                for item in report['report']['bs']:
                    itemUpper = item['label'].upper()
                    if item['concept'] == "us-gaap_Assets" and (item['label'].upper() == "TOTAL ASSETS"):
                        return item['value']
                    elif item['concept'] == "us-gaap_Assets" and item['label'].upper() == "ASSETS" and len(item['unit']) > 5:
                        return item['value']
            else:
                continue
    else:
        return None
  # Step 1: use the finnhub API to obtain "Financials As Reported"
  # Please pay attention to the API rate limit. I recommend you to let the program sleep for one second after you make an API call
  
  # Step 2: parse the financials returned from the finnhub API to get the total assets in a specific year-quarter
  # Total assets is reported in the balance sheets ('bs'). Total assets of a company is the value of the balance sheet item with its concept == "us-gaap_Assets"
  # Also, a few exceptions may occur in this process: 
  # (1) 'data' is not a key in the returned dictionary;
  # (2) the API does not return data for the specific year-quarter being queried;
  # (3) the financial data in the specific year-quarter does not have total assets (i.e., no balance sheet items have its concept == "us-gaap_Assets")
  
  # You can use if statements to handle these exceptions
  # Please listen to the lecture to get more details

print(get_quarterly_total_assets(API_KEY, "iep", 2025, 1))

15481000000.0


In [203]:
def get_peers_quarterly_total_assets(api_key, company, year, quarter):
  empty_list = []
  # Step 1: use the finnhub API to obtain the subIndustry peers of a focal company
  peers = requests.get("https://finnhub.io/api/v1/stock/peers", params= {'symbol':company,'grouping':'industry','token':api_key}).json()
  for peer in peers:
      #print(peer)
      total_assets_data = get_quarterly_total_assets(api_key, peer, year, quarter)
      empty_list.append([peer, total_assets_data, year, quarter])
  #sorted_list = sorted(empty_list, key=lambda x: x[0])
  return print(empty_list)
  # Step 2: for every peer company (including the focal company in the peers returned by the finnhub API)
  # we obtain its total assets in a specific year-quarter by calling the get_quarterly_total_assets function

  # Step 3: write the peer, its total assets in a specific year-quarter, year and quarter to a csv file. The csv file is named with {focal company name}.csv
  # For Step 3: learn how to write rows to a csv file from https://www.freecodecamp.org/news/how-to-create-a-csv-file-in-python/
  # For Step 3: you do not need to write a header in the csv file. Please see Canvas for example output files
get_peers_quarterly_total_assets(API_KEY, "aapl", 2025, 1)

[['AAPL', None, 2025, 1], ['DELL', None, 2025, 1], ['WDC', None, 2025, 1], ['PSTG', None, 2025, 1], ['HPE', None, 2025, 1], ['SMCI', None, 2025, 1], ['HPQ', None, 2025, 1], ['SNDK', None, 2025, 1], ['NTAP', None, 2025, 1], ['IONQ', None, 2025, 1], ['QUBT', None, 2025, 1], ['DBD', None, 2025, 1]]


In [ ]:
# combine results from all requests
with open('./Assignment2/company_symbols.csv', 'r') as f:
    # loop through each row
    # the delimiter is comma
    for line in f:
      focal_company = line.strip() # each line read from the csv file includes a newline character "\n" at the end, so we use .strip() to strip the newline character from the end of the line

      # call the function get_peers_quarterly_total_assets to obtain the total assets of the focal company and its peers in a specific year-quarter (2025-Q1 for this assignment)
      
      get_peers_quarterly_total_assets(api_key = API_KEY,
                                  company = focal_company,
                                  year = 2025,
                                  quarter = 1)
